In [29]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

label = pd.read_pickle('label.pkl')
label.index[0]
label = label.loc['2021-01-01 00:00:00':]
label = label.rename(columns={'value':'week'})


In [9]:
from tsfresh.feature_extraction import extract_features
from tsfresh.feature_extraction import EfficientFCParameters

def FeatureExtractor(data, window_size, fc_param=EfficientFCParameters()):
    data['id'] = np.ones((window_size,1), dtype=np.int8)
    data['time'] = np.arange(window_size)
    result = extract_features(data, column_id="id", column_sort="time", default_fc_parameters=fc_param)
    return result

def GetFeatureMatrix(data, window_size):
    feature_matrix = pd.DataFrame()
    for index in range(len(data)):
        if index < window_size:
            roll_label = data[:index+1]
        else:
            roll_label = data[index-window_size:index+1]
        feature_row = FeatureExtractor(roll_label, window_size=len(roll_label))
        feature_matrix = pd.concat([feature_matrix, feature_row], ignore_index=True)
    return feature_matrix

window_size = 24*7
feature_matrix = GetFeatureMatrix(label, window_size)

Feature Extraction: 100%|██████████| 1/1 [00:10<00:00, 10.63s/it]


In [10]:
# feature_matrix.to_pickle('2021.pkl')

In [17]:
feature_matrix_processed = feature_matrix.shift(1)
feature_matrix_processed['date'] = label.index
feature_matrix_processed = feature_matrix_processed.set_index('date', drop=True)
feature_matrix_processed = feature_matrix_processed[1:]

,value__variance_larger_than_standard_deviation,value__has_duplicate_max,value__has_duplicate_min,value__has_duplicate,value__sum_values,value__abs_energy,value__mean_abs_change,value__mean_change,value__mean_second_derivative_central,value__median,...,value__permutation_entropy__dimension_5__tau_1,value__permutation_entropy__dimension_6__tau_1,value__permutation_entropy__dimension_7__tau_1,value__query_similarity_count__query_None__threshold_0.0,"value__matrix_profile__feature_""min""__threshold_0.98","value__matrix_profile__feature_""max""__threshold_0.98","value__matrix_profile__feature_""mean""__threshold_0.98","value__matrix_profile__feature_""median""__threshold_0.98","value__matrix_profile__feature_""25""__threshold_0.98","value__matrix_profile__feature_""75""__threshold_0.98"
date,,,,,,,,,,,,,,,,,,,,,
2021-01-01 01:00:00,0.0,0.0,0.0,0.0,50.87,2587.7569,NaN,NaN,NaN,50.870,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 02:00:00,1.0,0.0,0.0,0.0,99.06,4910.0330,2.680000,-2.680000,NaN,49.530,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 03:00:00,1.0,0.0,0.0,0.0,143.74,6906.3354,3.095000,-3.095000,-0.415000,48.190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 04:00:00,1.0,0.0,0.0,0.0,186.66,8748.4618,2.650000,-2.650000,0.230000,46.435,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 05:00:00,1.0,0.0,0.0,0.0,227.05,10379.8139,2.620000,-2.620000,0.025000,44.680,...,-0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-01 19:00:00,1.0,0.0,0.0,1.0,1420.55,81903.3947,3.634583,0.737083,-0.150652,56.550,...,2.648438,2.926418,2.944439,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-01 20:00:00,1.0,0.0,0.0,1.0,1445.89,85110.9319,3.330417,0.432917,0.069565,57.560,...,2.529295,2.857103,2.944439,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-01 21:00:00,1.0,0.0,0.0,1.0,1460.32,87211.5070,3.313333,0.415833,0.068478,57.560,...,2.400563,2.761626,2.871476,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
feature_matrix_processed.to_pickle('2021_features_week.pkl')